<a href="https://colab.research.google.com/github/CobaMasa/PythonKerasDeeplearning/blob/main/chapter8_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 8-3: Style transfer with NN (Neural Style Transfer)

In [1]:
# change the version
!pip install tensorflow==1.5.0
!pip install keras==2.0.8

In [3]:
# List 8-14&15: Define variables
import numpy as np

from keras.preprocessing.image import load_img, img_to_array
from keras.applications import vgg19

# Set paths
target_image_path = 'drive/MyDrive/Study/PythonKeras/images/cat.jpg'
style_reference_image_path = 'drive/MyDrive/Study/PythonKeras/images/monet.jpg'

# Set image size 
width, height, = load_img(target_image_path).size
img_height = 800
img_width = int(width*img_height/height)

# def some compimentaly functions
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_height, img_width))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

def deprocess_image(x):
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68

    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

Using TensorFlow backend.


In [4]:
# List 8-16: Get ready to apply the model to images
import keras.backend as K

target_image = K.constant(preprocess_image(target_image_path))
style_reference_image = K.constant(preprocess_image(style_reference_image_path))
combination_image = K.placeholder((1, img_height, img_width, 3))

input_tensor = K.concatenate([target_image, style_reference_image, combination_image], axis=0)

model  = vgg19.VGG19(input_tensor=input_tensor, weights='imagenet', include_top=False)

In [5]:
# List 8-17~20: define contents & style loss functions

# loss for contents
def content_loss(base, combination):
    return K.sum(K.square(combination-base))

# loss for style
def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

def style_loss(style, combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_height * img_width
    return K.sum(K.square(S-C)) / (4. * (channels**2) * (size**2))

# whole loss
def total_variation_loss(x):
    a = K.square(x[:, :img_height-1, :img_width-1, :] - x[:, 1:, :img_width-1, :])
    b = K.square(x[:, :img_height-1, :img_width-1, :] -x[:, :img_height-1, 1:, :])
    return K.sum(K.pow(a+b, 1.25))

In [6]:
# List 8-20
outputs_dict = dict([(layer.name, layer) for layer in model.layers])
content_layer = 'block5_conv2'
style_layers = ['block{}_conv1'.format(i) for i in range(1, 6)]

# add contents loss weight
total_variation_weight = 1e-4
style_weight = 1.
content_weight = 0.025

loss = K.variable(0.)

layer_features = outputs_dict[content_layer]
target_image_features = layer_features.output[0, :, :, :]
combination_features = layer_features.output[2, :, :, :]
loss = loss + content_weight * content_loss(target_image_features,
                                      combination_features)
for layer_name in style_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features.output[1, :, :, :]
    combination_features = layer_features.output[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features)
    loss += (style_weight / len(style_layers)) * sl
loss = loss + total_variation_weight * total_variation_loss(combination_image)

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [8]:
# List 8-21: Define Gradient discendent process
grads = K.gradients(loss, combination_image)[0]

# Function to fetch the values of the current loss and the current gradients
fetch_loss_and_grads = K.function([combination_image], [loss, grads])


class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        x = x.reshape((1, img_height, img_width, 3))
        outs = fetch_loss_and_grads([x])
        loss_value = outs[0]
        grad_values = outs[1].flatten().astype('float64')
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

evaluator = Evaluator()

In [ ]:
# List 8-22: Style transfer loop
from scipy.optimize import fmin_l_bfgs_b
import imageio # substitute scipy.misc.imsave to this
import time

result_prefix = 'style_transfer_result'
iterations = 20

# Run scipy-based optimization (L-BFGS) over the pixels of the generated image
# so as to minimize the neural style loss.
# This is our initial state: the target image.
# Note that `scipy.optimize.fmin_l_bfgs_b` can only process flat vectors.
x = preprocess_image(target_image_path)
x = x.flatten()
for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x,
                                     fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    # Save current generated image
    img = x.copy().reshape((img_height, img_width, 3))
    img = deprocess_image(img)
    fname = result_prefix + '_at_iteration_%d.png' % i
    imageio.imwrite(fname, img) # substitute scipy.misc.imsave to this
    end_time = time.time()
    print('Image saved as', fname)
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

Start of iteration 0
Current loss value: 1929732600.0
Image saved as style_transfer_result_at_iteration_0.png
Iteration 0 completed in 876s
Start of iteration 1
Current loss value: 1205489900.0
Image saved as style_transfer_result_at_iteration_1.png
Iteration 1 completed in 849s
Start of iteration 2
Current loss value: 951356300.0
Image saved as style_transfer_result_at_iteration_2.png
Iteration 2 completed in 843s
Start of iteration 3
Current loss value: 777369900.0
Image saved as style_transfer_result_at_iteration_3.png
Iteration 3 completed in 843s
Start of iteration 4
Current loss value: 659198700.0
Image saved as style_transfer_result_at_iteration_4.png
Iteration 4 completed in 878s
Start of iteration 5
Current loss value: 571629500.0
Image saved as style_transfer_result_at_iteration_5.png
Iteration 5 completed in 857s
Start of iteration 6


## VAE: Valiational Auto Encoder

In [10]:
# List 8-23: VAE networks
import keras
from kears.layers import Input, Conv2D, Flatten, Dense, Lambda
from keras.models import Models
from keras import backend as K
import numpy as np

img_shape = (28, 28, 1)
batch_size = 16
latent_dim = 2

input_img = Input(shape=(img_shape))
x = Conv2D(32, 3, padding='same', activation='relu')(input_img))
x = Conv2D(64, 3, padding='same', activation='relu', stride=(2, 2))(x)
x = Conv2D(64, 3, padding='same', activation='relu'(x)
x = Conv2D(64, 3, padding='same', activation='relu'(x)

shape_before_flattening = K.int_shape(x)

x = Flatten()(x)
x = Dense(32, activation='relu')(x)

z_mean = Dense(latent_dim)(x)
z_log_var = Dense(latent_dim)(x)

In [ ]:
# List 8-24: samping function from latent space
def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim), mean=0., stddev=1.)
    return z_mean + K.exp(z_log_var) * epsilon

z = Lambda(sampling)([z_mean, z_log_var])

In [ ]:
# List 8-25: VAE decoder network for mapping latent point to an image
decoder_input = Input(K.int_shape(z)[1:])
